In [15]:
import sys
import os
import json
import string
import pprint
import csv
import re
from datetime import datetime
import math

company_list = []
frequency = {}
with open('SP500.csv') as SP500csv:
    reader = csv.DictReader(SP500csv)
    for row in reader:
        company_list.append(row['Symbol'])

buy_keywords = []
sell_keywords = []
with open('keyword.csv') as keyword:
    reader = csv.DictReader(keyword)
    for row in reader:
        buy_keywords.append(row['Buy'])
        if row['Sell']:
            sell_keywords.append(row['Sell'])
        
#Create sentiment accumulation for different dates
day = 31
n_company = len(company_list)
acc_sentiment = [0] * n_company
for i in range(n_company):
    acc_sentiment[i] = [0] * day

path = os.path.join("Extra_Storage", "Archiveteam_Halfyear", "2016", "01")
#for run_date in range(1,30):
    #if run_date < 10:
    #    sub_path = os.path.join(path, "0" + str(run_date))
    #else:
    #    sub_path = os.path.join(path, str(run_date))
        
    #for dirPath, dirNames, fileNames in os.walk(sub_path):
for dirPath, dirNames, fileNames in os.walk(path):
        for file in fileNames:
            if file.endswith("json"):    
                    filepath = os.path.join(dirPath, file)
                    with open(filepath) as json_data:
                        data=[]                        
                        for line in json_data:
                            try:
                                data.append(json.loads(line))
                            except json.JSONDecodeError:
                                continue
                        for tweet in data:
                            if 'text' in tweet:
                                tweet_lower = tweet['text'].lower()
                                company_index = -1
                                for company in company_list:
                                    company_index+=1
                                    company_tag = ' $' + company + ' '
                                    if company_tag.lower() in tweet_lower:
                                        sentiment = 0                                    
                                        for buy_keyword in buy_keywords:
                                            if buy_keyword in tweet_lower:
                                                sentiment+=1
                                        for sell_keyword in sell_keywords:
                                            if sell_keyword in tweet_lower:
                                                sentiment-=1
                                        if sentiment==0:
                                            break
                                        else:
                                            if 'created_at' in tweet:
                                                second = (datetime.strptime(tweet['created_at'],'%a %b %d %H:%M:%S +%f %Y')- datetime(2016, 1, 1)).total_seconds()
                                                date = math.ceil(second/86400) - 1
                                                if date ==  len(acc_sentiment[0]):
                                                    date -= 1
                                            else: 
                                                break
                                            if sentiment > 0:
                                                acc_sentiment[company_index][date]+=1
                                            else:
                                                acc_sentiment[company_index][date]-=1

                                    
#                    match_pattern = re.findall(r'\b[a-z]{3,15}\b', all_tweet)
#                    for word in match_pattern:
#                        count = frequency.get(word,0)
#                        frequency[word] = count + 1
                        
#sorted_frequency = sorted(frequency.items(), key=lambda frequency: frequency[1], reverse=True)
#i = 0
#for words in sorted_frequency:
#    if i < 100:
#        print(words)
#        i+=1
#    else:
#        break                    

#frequency_list = frequency.keys()
#for words in frequency_list:
#    print(words, frequency[words])

In [16]:
import numpy as np
import csv
#def save_sentiments_csv(acc_sentiment):
#Generate dates
dates = []
for thedate in range(1, day+1):
    dates.append('1/'+ str(thedate) + '/2016')

with open('acc_sentiment_2016_01_paper.csv', 'w', newline='') as outcsv:
    writer = csv.writer(outcsv)
    writer.writerow(dates)
    #arr = np.array(acc_sentiment)
    #np.savetxt('acc_sentiment_2016_04(2).csv', arr, fmt='%i', delimiter=',')
    writer.writerows(acc_sentiment)